# OBSOLETE BECAUSE DESTROYS FORMULAS

# Readme >>Loading and Writing Tables (JL to/from XL) [Preseving formulas][copilot]

## 📥 `load_df` — Read Excel Table into a Jupyter-Friendly DataFrame

## 📤 `write_df` — Push Updated DataFrame Back to Excel as a Table

This function will:

- Overwrite the `xl_tbl` worksheet
- Rebuild the Excel table with your edited DataFrame
- Preserve column names and structure


To use it:

```python
write_df(df_tbl)
```

---

## 🧠 Extendability

You can swap `"tbl"` for any table name like this:

```python
df_hr = load_df("hr_tbl", "hr_tbl.xlsx", "hr_tbl")
# edit df_hr...
write_df(df_hr, "hr_tbl", "hr_tbl.xlsx", "hr_tbl")
```

---

# *def* load_df_with_formulas(file_name="xl_tbl.xlsx", sheet_name="xl_tbl"): *>>return df*

In [5]:
from openpyxl import load_workbook
import pandas as pd

def load_df_with_formulas(file_name="xl_tbl.xlsx", sheet_name="xl_tbl"):
    wb = load_workbook(file_name, data_only=False)  # `data_only=False` preserves formulas
    ws = wb[sheet_name]

    data = []
    for row in ws.iter_rows(values_only=False):
        row_data = []
        for cell in row:
            if cell.value is not None:
                if cell.data_type == 'f':  # It's a formula
                    row_data.append(f"={cell.value}")
                else:
                    row_data.append(cell.value)
            else:
                row_data.append(None)
        data.append(row_data)

    # Extract headers from first row
    headers = data[0]
    df = pd.DataFrame(data[1:], columns=headers)
    print(f"✅ Loaded sheet '{sheet_name}' from '{file_name}', preserving formulas.")
    return df


# *def* write_df_with_formulas(df, file_name="xl_tbl.xlsx", sheet_name="xl_tbl", table_name="xl_tbl"): *>> return df*

In [6]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def write_df_with_formulas(df, file_name="xl_tbl.xlsx", sheet_name="xl_tbl", table_name="xl_tbl"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheet_name

    # Write headers
    for col_idx, col_name in enumerate(df.columns, start=1):
        ws.cell(row=1, column=col_idx, value=col_name)

    # Write values or formulas
    for row_idx, row in enumerate(df.itertuples(index=False), start=2):
        for col_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=row_idx, column=col_idx)
            if isinstance(value, str) and value.startswith("="):
                cell.value = value[1:]  # Remove leading '=' for Excel formula
                cell.data_type = 'f'
            else:
                cell.value = value

    # Define table
    nrows, ncols = df.shape
    table_range = f"A1:{get_column_letter(ncols)}{nrows+1}"

    from openpyxl.worksheet.table import Table, TableStyleInfo
    table = Table(displayName=table_name, ref=table_range)
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
    table.tableStyleInfo = style
    ws.add_table(table)

    wb.save(file_name)
    print(f"💾 Wrote updated DataFrame to Excel file '{file_name}', preserving formulas.")
